In [17]:
# Heat budget terms for the different layers
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os

In [18]:
sd = 150 # sill depth

state01 = xr.open_dataset('/Users/weiyangbao/Documents/Model_outputs/sill_drag/state_' + str(format(sd,'03d')) + '.nc')
state02 = xr.open_dataset('/Users/weiyangbao/Documents/Model_outputs/sill_drag/Adv_' + str(format(sd,'03d')) + '.nc')
state03 = xr.open_dataset('/Users/weiyangbao/Documents/Model_outputs/sill_drag/Dif_' + str(format(sd,'03d')) + '.nc')

state1 = state01.isel(Y=range(1,10), T=range(40,121))
state2 = state02.isel(Y=range(1,10), T=range(39,120))
state3 = state03.isel(Y=range(1,10), T=range(39,120))

Area = np.empty([50, 9]) # Cross Y direction
Area[:10, :] = 100
Area[10:20, :] = 200
Area[20:30, :] = 300
Area[30:40, :] = 800
Area[40:, :] = 1600

T0 = 9 # Initial temperature
f = 3 # Control volume start index near the ice front
s = 28 # Control volume end index at the sill crest
Cp = 3994  # Specific heat content J/kg/C
rho = 999.8 # Density

In [19]:
# Read data in T,Z,Y,X
pt = state1.Temp.data

u0 = state1.U.data
u = 0.5 * (u0[:,:,:,:-1] + u0[:,:,:,1:])

w0 = state1.W.data
w = 0.5 * (w0[:,:-1,:,:] + w0[:,1:,:,:])

upt = u * (pt-T0)

advx = state2.ADVx_TH.data * Cp * rho
difx = state3.DFxE_TH.data * Cp * rho
advr = state2.ADVr_TH.data * Cp * rho
difr = state3.DFrI_TH.data * Cp * rho
#dfx0 = state2.DFxE_TH.data
#dfx = 0.5 * (dfx0[:,:,:,:-1] + dfx0[:,:,:,1:])

#dfz0 = state3.DFrI_TH.data
#dfz = 0.5 * (dfz0[:,:-1,:,:] + dfz0[:,1:,:,:])


# Advective flux of potential temperature
ADf = np.sum(Area * upt[:,:,:,f].mean(0)) * Cp * rho 
ADs = -np.sum(Area * upt[:,:,:,s].mean(0)) * Cp * rho 

# Diffusive flux of potential temperature
DFf = np.sum(dfx[:,:,:,f].mean(0)) * Cp * rho
DFs = -np.sum(dfx[:,:,:,s].mean(0)) * Cp * rho 

"{:.2E}".format(ADf), "{:.2E}".format(ADs), "{:.2E}".format(DFf), "{:.2E}".format(DFs), "{:.2E}".format(ADf+DFf+ADs+DFs)

In [20]:
# Advective flux of potential temperature
ADf = np.sum(advx[:,:,:,f].mean(0))
ADs = -np.sum(advx[:,:,:,s].mean(0))

# Diffusive flux of potential temperature
DFf = np.sum(difx[:,:,:,f].mean(0))
DFs = -np.sum(difx[:,:,:,s].mean(0))
"{:.2E}".format(ADf), "{:.2E}".format(ADs), "{:.2E}".format(DFf), "{:.2E}".format(DFs), "{:.2E}".format(ADf+DFf+ADs+DFs)

('1.44E+08', '5.45E+08', '-7.16E+08', '2.86E+07', '1.44E+06')

In [21]:
## Budget terms for different layers

uzx = u0.mean(axis=(0,2)) # Z, X
# Time average
wt = w.mean(0) # Z, Y, X
#dfzt = dfz.mean(0)
ptt = pt.mean(0)
dxdy = 200**2

# Find the interface and vertical flux of theta
def Layer(uzx):
    # Layer interface, zero velocity crossing point
    X = uzx.shape[1]
    ADzx = np.empty(X)
    DFzx = np.empty(X)
    itf = np.empty(X, dtype=int)
    ADzx[[0,1,-2,-1]] = 0
    DFzx[[0,1,-2,-1]] = 0
    itf[[0,1,-2,-1]] = 0
    for i in range(2,X-2):
        l = np.where(uzx[:, i] < 0)[-1][0]
        itf[i] = l
        #ADzx[i] = np.sum(wt[l,:,i] * (ptt[l,:,i]-T0) * dxdy ) # Vertical advective flux through the interface
        #DFzx[i] = np.sum(dfz[:,l,:,i].mean(0)) # Vertical diffusive flux through the interface
        ADzx[i] = np.sum(advr[:,l,:,i].mean(0))
        DFzx[i] = np.sum(difr[:,l,:,i].mean(0))
    return (itf, ADzx, DFzx)

itf, ADzx, DFzx = Layer(uzx)

ADz = ADzx[f:s+1].sum() * Cp * rho
DFz = DFzx[f:s+1].sum() * Cp * rho 


ADf1 = np.sum(Area[:itf[f],:] * upt[:,:itf[f],:,f].mean(0)) * Cp * rho 
ADs1 = -np.sum(Area[:itf[s],:] * upt[:,:itf[s],:,s].mean(0)) * Cp * rho 
DFf1 = np.sum(dfx[:,:itf[f],:,f].mean(0)) * Cp * rho 
DFs1 = -np.sum(dfx[:,:itf[s],:,s].mean(0))  


ADf2 = np.sum(Area[itf[f]:,:] * upt[:,itf[f]:,:,f].mean(0)) * Cp * rho
ADs2 = -np.sum(Area[itf[s]:,:] * upt[:,itf[s]:,:,s].mean(0)) * Cp * rho
DFf2 = np.sum(dfx[:,itf[f]:,:,f].mean(0))
DFs2 = -np.sum(dfx[:,itf[s]:,:,s].mean(0)) 

ptz = 0.5 * (ptt[itf[s]-1,:,f:s+1] + ptt[itf[s],:,f:s+1])
ADz1 = np.sum(wt[itf[s],:,f:s+1].mean(0) * (ptz-T0) * dxdy)
#ADz2 = np.sum(wt[itf[s],:,f:s+1] * (ptt[itf[s],:,f:s+1]-T0) * dxdy)
DFz1 = np.sum(dfz[:,itf[s],:,f:s+1].mean(0))

#ADf2 = np.sum(Area[itf[f]:,:] * upt[:,itf[f]:,:,f].mean(0))
#ADs2 = ADs - ADs1

"{:.2E}".format(DFf1), "{:.2E}".format(DFs1), "{:.2E}".format(ADz), "{:.2E}".format(DFz), "{:.2E}".format(ADf1+ADs1+DFf1+DFs1+ADz+DFz)

In [22]:
itf, ADzx, DFzx = Layer(uzx)
ADz = ADzx[f:s+1].sum() 
DFz = DFzx[f:s+1].sum() 

ADf1 = np.sum(advx[:,:itf[f],:,f].mean(0))
ADs1 = -np.sum(advx[:,:itf[s],:,s].mean(0))
DFf1 = np.sum(difx[:,:itf[f],:,f].mean(0))
DFs1 = -np.sum(difx[:,:itf[s],:,s].mean(0))

ADf2 = np.sum(advx[:,itf[f]:,:,f].mean(0))
ADs2 = -np.sum(advx[:,itf[s]:,:,s].mean(0))
DFf2 = np.sum(difx[:,itf[f]:,:,f].mean(0))
DFs2 = -np.sum(difx[:,itf[s]:,:,s].mean(0))


"{:.2E}".format(ADf1), "{:.2E}".format(ADs1), "{:.2E}".format(DFf1), "{:.2E}".format(DFs1)

('1.04E+11', '-1.12E+11', '-4.05E+08', '1.17E+07')

In [23]:
"{:.2E}".format(ADf2), "{:.2E}".format(ADs2), "{:.2E}".format(DFf2), "{:.2E}".format(DFs2)

('-1.03E+11', '1.13E+11', '-3.11E+08', '1.69E+07')

In [24]:
"{:.2E}".format(ADz), "{:.2E}".format(DFz), "{:.2E}".format(ADf2+ADs2+DFf2+DFs2-ADz-DFz),"{:.2E}".format(ADf1+ADs1+DFf1+DFs1+ADz+DFz)

('8.42E+09', '1.99E+08', '3.16E+08', '-3.14E+08')